In [141]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee
from scipy.optimize import curve_fit
from scipy.stats.distributions import chi2
from scipy.stats import multivariate_normal
from os import listdir
import matplotlib as mpl
%matplotlib inline

new_rc_params = {'text.usetex': False,
    "svg.fonttype": 'none'
    }
mpl.rcParams.update(new_rc_params)

In [2]:
#Voting history from all 51 'states' since 1976
votinghistory_full = np.loadtxt('1976-2020-president.csv', delimiter=',', skiprows=1, usecols=(0, 1, 10, 11, 14), 
                        dtype=[('year', int), ('state', 'U25'), ('votes', int), ('total_votes', int), ('party', 'U20')])
print(votinghistory_full)

[(1976, 'ALABAMA', 659170, 1182850, 'DEMOCRAT')
 (1976, 'ALABAMA', 504070, 1182850, 'REPUBLICAN')
 (1976, 'ALABAMA',   9198, 1182850, 'OTHER') ...
 (2020, 'WYOMING',   1739,  278503, 'OTHER')
 (2020, 'WYOMING',    279,  278503, 'OTHER')
 (2020, 'WYOMING',   1459,  278503, 'OTHER')]


In [3]:
#Voting results from only Democratic and Republican candidates
index_dem = votinghistory_full['party'] == 'DEMOCRAT'
index_rep = votinghistory_full['party'] == 'REPUBLICAN'
votinghistory_dem = votinghistory_full[index_dem] # democrat results only
votinghistory_rep = votinghistory_full[index_rep] # republican results only
votinghistory = np.concatenate([votinghistory_dem, votinghistory_rep]) # combined results

In [4]:
#Array of states in alphabetical order
states = np.unique(votinghistory['state'])

In [119]:
# Now, considering all races between 1976 and 2016, we calculate the mean voteshare and variance for each state,
# for each party, along with the full covariance matrix

#Democrats
index_dem_2016 = votinghistory_dem['year'] != 2020
years = np.unique(votinghistory_dem[index_dem_2016]['year'])
votinghistory_dem_2016 = votinghistory_dem[index_dem_2016]
voteshare_dem = []
for i in range(len(states)):
    state = states[i]
    index = votinghistory_dem_2016['state'] == str(state)
    array = votinghistory_dem_2016[index]
    out = []
    for j in range(len(years)):
        voteshare = array[j][2] / array[j][3]
        out.append(voteshare)
    voteshare_dem.append(out)
cov_dem = np.cov(voteshare_dem, bias = True, dtype=np.float64)
var_dem = np.diag(cov_dem)
mean_dem = np.mean(voteshare_dem, axis = 1)

#Republicans
index_rep_2016 = votinghistory_rep['year'] != 2020
years = np.unique(votinghistory_rep[index_rep_2016]['year'])
votinghistory_rep_2016 = votinghistory_rep[index_rep_2016]
voteshare_rep = []
for i in range(len(states)):
    state = states[i]
    index = votinghistory_rep_2016['state'] == str(state)
    array = votinghistory_rep_2016[index]
    out = []
    for j in range(len(years)):
        voteshare = array[j][2] / array[j][3]
        out.append(voteshare)
    voteshare_rep.append(out)
cov_rep = np.cov(voteshare_rep, bias = True)
var_rep = np.diag(voteshare_rep)
mean_rep = np.mean(voteshare_rep, axis = 1)

In [6]:
# Ingesting polling data from 2020 election
polling_averages = np.loadtxt('presidential_general_averages.csv', delimiter=',', skiprows = 1, usecols = (0, 1, 2, 3), 
                              dtype=[('candidate', 'U20'), ('date', 'U10'), ('approval', float), ('state', 'U25')])

index_biden = polling_averages['candidate'] == 'Joseph R. Biden Jr.'
index_trump = polling_averages['candidate'] == 'Donald Trump'
polls_biden = polling_averages[index_biden]
polls_trump = polling_averages[index_trump]

In [139]:
# Organizing polling data by state to establish covariance matrix for multivariate normal likelihood

states_lowercase = np.unique(polling_averages['state'])
polling_dates = np.unique(polling_averages['date'])

# Joe Biden
polls_biden_bystate = []
polls_biden_national = []
for i in range(len(states_lowercase)):
    state = states_lowercase[i]
    index = polls_biden['state'] == str(state)
    array = polls_biden[index]
    mean = np.mean(array['approval'])
    stdv = np.std(array['approval'])
    if state == 'ME-1':
        continue
    if state == 'ME-2':
        continue
    if state == 'NE-2':
        continue
    if state == 'National':
        polls_biden_national.append(array)
    else:
        out = []
        for j in range(len(polling_dates)):
            date = polling_dates[j]
            datapoint = array[array['date'] == str(date)]
            if not datapoint.tolist():
                new_datapoint = np.random.normal(mean, stdv) * 0.01
                out.append(new_datapoint)
            else:
                out.append(datapoint['approval'][0] * 0.01)
        polls_biden_bystate.append(out)
polls_biden_cov = np.cov(polls_biden_bystate, bias = True)

# Donald Trump
polls_trump_bystate = []
polls_trump_national = []
for i in range(len(states_lowercase)):
    state = states_lowercase[i]
    index = polls_trump['state'] == str(state)
    array = polls_trump[index]
    mean = np.mean(array['approval'])
    stdv = np.std(array['approval'])
    if state == 'ME-1':
        continue
    if state == 'ME-2':
        continue
    if state == 'NE-2':
        continue
    if state == 'National':
        polls_trump_national.append(array)
    else:
        out = []
        for j in range(len(polling_dates)):
            date = polling_dates[j]
            datapoint = array[array['date'] == str(date)]
            if not datapoint.tolist():
                new_datapoint = np.random.normal(mean, stdv) * 0.01
                out.append(new_datapoint)
            else:
                out.append(datapoint['approval'][0] * 0.01)
        polls_trump_bystate.append(out)
polls_trump_cov = np.cov(polls_trump_bystate, bias = True)

print(len(np.asarray(polls_trump_bystate).T))
print(np.asarray(polls_trump_bystate).T[3])

251
[0.5923509  0.50534504 0.4820326  0.44748328 0.3435703  0.4408042
 0.3474097  0.4051353  0.10199435 0.4760153  0.4825122  0.30697724
 0.57478079 0.38912986 0.51730346 0.4897333  0.5228951  0.55866137
 0.51650424 0.40271137 0.3480251  0.30728282 0.4443297  0.42906497
 0.5722068  0.5091909  0.5609013  0.52590837 0.4201992  0.460477
 0.3593914  0.474118   0.37106    0.4631087  0.5697821  0.4642032
 0.6310739  0.39088811 0.4537696  0.33134947 0.5103901  0.55734211
 0.5618425  0.4921024  0.5099411  0.31105542 0.4300517  0.35773874
 0.6652835  0.46467    0.63863837]


In [268]:
# Implementing MCMC
# Prior
def prior(mu, mu_0, sigma_0):
    return multivariate_normal.pdf(mu, mu_0, sigma_0, allow_singular = True)

# Likelihood
def likelihood(mu, polls, sigma):
    polls_l = np.asarray(polls).T
    index = len(polls_l)
    probs = []
    for i in range(index):
        prob = multivariate_normal.pdf(polls_l[i], mu, sigma, allow_singular = True)
        probs.append(prob)
    return np.prod(np.asarray(probs))

# Posterior
def log_post(mu, polls, sigma, mu_0, sigma_0):
    like = likelihood(mu, polls, sigma)
    pri = prior(mu, mu_0, sigma_0)
    post = like * pri
    if post > 0.:
        return np.log(post)
    else: return -np.inf

In [ ]:
# MCMC
num_iter = 5000
ndim = 51
nwalkers = ndim * 2
initial_pos = np.array(mean_dem) + 0.01 * np.random.randn(nwalkers, ndim)
print(initial_pos)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_post, 
                                args=[polls_biden_bystate, polls_biden_cov, mean_dem, cov_dem])
sampler.run_mcmc(initial_pos, num_iter, progress=True, skip_initial_state_check = True);

[[0.41794091 0.33435935 0.40611049 ... 0.45694596 0.49008815 0.32605162]
 [0.42512532 0.33305696 0.42865929 ... 0.43257731 0.50041751 0.32091332]
 [0.42180555 0.34215004 0.41044476 ... 0.45728547 0.49697927 0.31916513]
 ...
 [0.42645428 0.31913002 0.39984267 ... 0.45530687 0.487565   0.30514166]
 [0.41539882 0.3445717  0.40193231 ... 0.43530501 0.48260635 0.29910642]
 [0.41344264 0.34058636 0.41677021 ... 0.47588255 0.48672035 0.32046706]]


  0%|          | 1/5000 [00:09<13:22:37,  9.63s/it]